# Web of Science
- 학술 데이터베이스 Web of Science에서 자료를 수집하는 코드 구현  
- 학내 정책상 자동화된 프로그램을 이용해서 학술 데이터에 접근하는 것은 서버 과부화를 야기하기 때문에 무분별하게 사용하면 문제발생
(사용하지 않는 것을 권고, 사용하는 경우 도서관 사이트관리자와 충분히 상의하기)  
- 작업에 앞서 본인이 학술 데이터베이스를 사용하는 과정에 대해 고민

### [작업과정]  
(1) 경희대학교 중앙도서관 접속 및 로그인  
(2) 학술 데이터베이스 로그인  
(3) 검색어 검색  
(4) 카테고리 선택  
(5) paper 수집  

In [34]:
from selenium import webdriver
from time import sleep
from random import randint
from tqdm import tqdm
import time
import pandas as pd
import sys

In [9]:
# print(sys.path)

In [10]:
sys.path.append('/Users/jiwonjang/python_tutorial/ve_env/') # 시스템 경로 추가

In [4]:
# for i in sys.path:
#     print(i)

In [11]:
import info21

In [12]:
info21ID = info21.infoID # info21 id
info21PW = info21.infoPassward # info21 password

# Web of Science 접속

In [13]:
### access 'web of science'
chromeDir = '/Users/jiwonjang/python_tutorial/chromedriver' # driver directory
driver = webdriver.Chrome(chromeDir)
driver.implicitly_wait(1)

In [14]:
# connect khu library
univ_url = 'https://external.khu.ac.kr/sns/login.php?return_url=https%3A%2F%2Flibrary.khu.ac.kr%2Fseoul&campus=seoul'
driver.get(univ_url) 
time.sleep(1)

In [15]:
# khu library login
driver.find_element_by_xpath('//*[@id="inputEmail3"]').send_keys(info21ID) 
driver.find_element_by_xpath('//*[@id="inputPassword3"]').send_keys(info21PW)
driver.find_element_by_xpath('/html/body/div/div[1]/div/div/form/div[3]/div/button').click()

In [16]:
# connect web of science 
driver.get('http://access.openlink.khu.ac.kr/link.n2s?url=http://isiknowledge.com/wos')

In [17]:
driver.current_url

'http://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=E5UV1VWnPVHpllSChAp&preferencesSaved='

# Query 검색

In [18]:
sid_id = driver.find_element_by_id('SID').get_attribute('value') # url 구성을 위한 sid id 생성
print('sid id:', sid_id)

sid id: E5UV1VWnPVHpllSChAp


In [19]:
time.sleep(2)
query = 'reinforcement learning'
print(query)
driver.find_element_by_id('value(input1)').send_keys(query)
time.sleep(1)

reinforcement learning


In [20]:
driver.find_element_by_xpath('//*[@id="searchCell1"]/span[1]/button').click() # query click

In [21]:
# go to the category page
WOS_category_url = 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/RAMore.do?product=WOS&search_mode=GeneralSearch&' + sid_id + '&qid=1&ra_mode=more&ra_name=JCRCategories&colName=WOS&viewType=raMore'
driver.get(WOS_category_url)

In [22]:
# 관심항목 체크
driver.find_element_by_xpath('//*[@id="TASCA_2"]').click() # computer science

In [23]:
# 관심항목 체크 후 하단 범위재설정 클릭
driver.find_element_by_xpath('//*[@id="raMore"]/table/tbody/tr[1]/td/table/tbody/tr/td[2]/div/button').click()

In [24]:
print(driver.current_url)

http://apps.webofknowledge.com/Search.do?product=WOS&SID=E5UV1VWnPVHpllSChAp&search_mode=GeneralSearch&prID=92a2945d-0b4e-4d1a-bbd3-8a529f8a9001


In [25]:
# 순회할 수 있는 url을 얻기 위해 다음 페이지 선택
driver.find_element_by_xpath('//*[@id="summary_navigation"]/nav/table/tbody/tr/td[3]/a').click()

In [26]:
print(driver.current_url)

http://apps.webofknowledge.com/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=E5UV1VWnPVHpllSChAp&colName=WOS&&update_back2search_link_param=yes&page=2


# 논문 수집

In [27]:
base_url = driver.current_url[:-1] # 기존 url 도출

In [28]:
print(base_url)

http://apps.webofknowledge.com/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=E5UV1VWnPVHpllSChAp&colName=WOS&&update_back2search_link_param=yes&page=


In [29]:
New_post_list = []
pageNumber = 2
for i in range(1, pageNumber+1): 
    new_post_list_url = base_url + str(i)
    New_post_list.append(new_post_list_url)

In [30]:
New_post_list

['http://apps.webofknowledge.com/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=E5UV1VWnPVHpllSChAp&colName=WOS&&update_back2search_link_param=yes&page=1',
 'http://apps.webofknowledge.com/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=E5UV1VWnPVHpllSChAp&colName=WOS&&update_back2search_link_param=yes&page=2']

In [31]:
Paper_href_list = []
for new_url in New_post_list:
    driver.get(new_url)
    sleep(randint(1,3))
    
    paper_hrefs = driver.find_elements_by_css_selector('div.search-results-content > div:nth-child(1) > div > a')
    
    for href in paper_hrefs:
        paper_title_url = href.get_attribute('href')
        Paper_href_list.append(paper_title_url)

print('page search finish ----------------------------------------------')

page search finish ----------------------------------------------


In [32]:
Paper_href_list

['http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=1',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=2',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=3',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=4',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=5',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=6',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=E5UV1VWnPVHpllSChAp&page=1&doc=7',
 'http://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSear

In [33]:
len(Paper_href_list)

20

# collect paper data 
- 각 논문에 해당하는 정보 수집

In [ ]:
Paper_result_DF = pd.DataFrame(columns=['paper_title', 'publish_data', 'keyword', 'plus_keyword'], index=[])

for url in tqdm(Paper_href_list): # tqdm 추가
    keyword_list = []
    plus_keyword_list = []
    
    sleep(randint(1,3))
    driver.get(url)
    keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(2) > a')
    plus_keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(3) > a')
    
    for keyword in keyword_fetch:
#         print(keyword.text)
        keyword_list.append(keyword.text)
        
    for plus_keyword in plus_keyword_fetch:
#         print(plus_keyword.text)
        plus_keyword_list.append(plus_keyword.text)
        
    paper_title = driver.find_element_by_css_selector('div.title > value').text
#     print(paper_title)
    publish_data = driver.find_element_by_css_selector('div.block-record-info.block-record-info-source').text
#     print(publish_data)
    paper_result = {'paper_title' : paper_title, 'publish_data' : publish_data, 'keyword' : keyword_list, 'plus_keyword' : plus_keyword_list}
    Paper_result_DF = Paper_result_DF.append(paper_result, ignore_index=True)
    
print('finished!!!!')
# driver.close()

 25%|██▌       | 5/20 [00:17<00:52,  3.50s/it]

In [ ]:
driver.close() # driver 종료

In [68]:
Paper_result_DF.shape

(20, 4)

In [69]:
Paper_result_DF.head()

,paper_title,publish_data,keyword,plus_keyword
0,Emotional editing constraint conversation cont...,INFORMATION FUSION\n권: 56 페이지: 70-80\nDOI: 10....,"[, Emotional conversation generation, Affectiv...","[, , ]"
1,Application of deep reinforcement learning to ...,EXPERT SYSTEMS WITH APPLICATIONS\n권: 141\n논문 번...,"[, Intrusion detection, Data networks, Deep re...","[, INTERNET, SYSTEMS, NETWORK, , ]"
2,Continuous control with Stacked Deep Dynamic R...,EXPERT SYSTEMS WITH APPLICATIONS\n권: 140\n논문 번...,"[, Reinforcement learning, Policy gradient, De...","[, , ]"
3,Time-driven feature-aware jointly deep reinfor...,EXPERT SYSTEMS WITH APPLICATIONS\n권: 140\n논문 번...,"[, Algorithmic trading, Deep reinforcement lea...","[, , ]"
4,Effective image super resolution via hierarchi...,NEUROCOMPUTING\n권: 374 페이지: 109-116\nDOI: 10.1...,"[, Image super-resolution, Edges extraction, D...","[, SUPERRESOLUTION, , ]"


In [29]:
# Paper_result_DF.to_csv('WOS_example_RL.csv', header=True, encoding='utf-8')

------------------------------------------------------------------------------

# 실습!
- Web of Science 추가 작업 : 저자 이름 수집, 인용 문헌(인용 횟수) 수집
- 구글 스칼라(google scholar) 수집 프로그램 작업 (bs version, selenium version)